In [1]:
%load_ext autoreload
%autoreload 2

## Data

In [1]:
from torch.utils.data import DataLoader, ConcatDataset, WeightedRandomSampler
from dataset import get_dataset, get_weighted_sampler, get_concat_dataset
import torch
import numpy as np
import matplotlib.pyplot as plt

input_size = 224
batch_size = 128
n_worker = 8

train_dataset, valid_dataset = get_dataset(input_size)
concat_dataset = get_concat_dataset()

# sample_freq = np.bincount(train_dataset.targets + valid_dataset.targets)
# sample_weight = np.array([1/sample_freq[x] for x in train_dataset.targets] + [1/sample_freq[x] for x in valid_dataset.targets])
# sample_weight = torch.from_numpy(sample_weight)
# sampler = WeightedRandomSampler(sample_weight.type('torch.DoubleTensor'), len(sample_weight)//2)

sampler = get_weighted_sampler()

train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, sampler = sampler, num_workers=n_worker)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=n_worker)

dataloaders = { 'train' : train_loader, 'valid' : valid_loader}

## Model

In [2]:
train_dataset.classes

['Metal', 'Paper', 'Paperpack', 'Plastic', 'Plasticbag', 'Styrofoam']

In [9]:
import torch
import torchvision
from model import CustomVGG
import geffnet

cfg = [[int(64*0.75)], [int(128*0.75)], [int(256*0.75), int(256*0.75)], [int(512*0.75), int(512*0.75)], [int(512*0.75), 512]]
device = 'cuda'

# model = CustomVGG(cfg=cfg, bias=True)
# model.load_state_dict(torch.load('save/pruned_final.pt'))

model = geffnet.create_model('mobilenetv3_large_100', pretrained=True, num_classes=6)
model.to(device)
print(model)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /opt/ml/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


=> Discarding pretrained classifier since num_classes != 1000
MobileNetV3(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): HardSwishMe()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64

In [3]:
[int(64*0.75)], [int(128*0.75)], [int(256*0.75), int(256*0.75)], [int(512*0.75), int(512*0.75)], [int(512*0.75), 512]

([48], [96], [192, 192], [384, 384], [384, 512])

## Criterion & Optimizer

In [10]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss

lr = 0.0001

criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
#lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30], gamma=0.1) # change trainer step
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', verbose=True, patience=5) # optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=0.000001)#

## Train

In [11]:
from trainer import Trainer

num_epoch = 100

trainer = Trainer('/opt/ml/code/save', seed=42)
trainer.train(model, dataloaders, criterion, optimizer, lr_scheduler, num_epoch, 10, save_name='test')

Epoch 1/100
----------


100%|██████████| 61/61 [00:23<00:00,  2.55it/s]


train Epoch: 1 Loss: 1.8770 Acc: 0.4997 F1: 0.4999


100%|██████████| 41/41 [00:15<00:00,  2.65it/s]


valid Epoch: 1 Loss: 1.4219 Acc: 0.5959 F1: 0.5449

Epoch 2/100
----------


...

valid Epoch: 62 Loss: 0.8484 Acc: 0.7975 F1: 0.7729

Epoch 63/100
----------


100%|██████████| 61/61 [00:23<00:00,  2.54it/s]


train Epoch: 63 Loss: 0.0266 Acc: 0.9933 F1: 0.9933


100%|██████████| 41/41 [00:15<00:00,  2.70it/s]

valid Epoch: 63 Loss: 0.8449 Acc: 0.7983 F1: 0.7732

Training complete!
Best f1 score 0.7768 at epoch 52



In [6]:
depth experiment

Training complete!
Best f1 score 0.7875 at epoch 31

Training complete!
Best f1 score 0.7701 at epoch 39

SyntaxError: invalid syntax (2963668720.py, line 1)

In [ ]:
Best f1 score 0.4860 at epoch 30